In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import numpy as np


# Load datasets into Pandas DataFrames
dataframes = {}
file_names = ['CPA1.xlsx', 'CPA2.xlsx', 'CPA3.xlsx', 'CPB1.xlsx', 'CPB2.xlsx', 'CPB3.xlsx', 'CPB4.xlsx', 'CPB5.xlsx', 'CPB6.xlsx', 'CPB7.xlsx']

for filename in file_names:
    dataframes[filename.split('.')[0]] = pd.read_excel(filename)

# Gabungkan semua dataset menjadi satu DataFrame tunggal
combined_data = pd.concat(dataframes.values(), ignore_index=True)

# Memisahkan fitur (X) dan label (y)
X = combined_data[['powLv1', 'powLv2', 'powLv3', 'powLv4', 'powLv5']]
y = combined_data['kelas']

# Inisialisasi LabelEncoder
label_encoder = LabelEncoder()
# Konversi label kelas menjadi format numerik
y_encoded = label_encoder.fit_transform(y)

# Fungsi untuk menghitung sensitivity dan specificity
def calculate_sensitivity_specificity(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn = cm[0, 0]
    fp = cm[0, 1]
    fn = cm[1, 0]
    tp = cm[1, 1]
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    return sensitivity, specificity

# Fungsi untuk melatih dan menguji model menggunakan SVM atau KNN
def train_and_evaluate_model(X_train, y_train, X_test, y_test, classifier):
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    sensitivity, specificity = calculate_sensitivity_specificity(y_test, y_pred)
    return accuracy, f1, sensitivity, specificity


# Contoh penggunaan untuk metode 10-fold cross-validation
def cross_validation(X, y, classifier):
    accuracies = cross_val_score(classifier, X, y, cv=10)
    f1_scores = cross_val_score(classifier, X, y, cv=10, scoring='f1_weighted')
    sensitivity_scores = []
    specificity_scores = []

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        cm = confusion_matrix(y_test, y_pred)
        tn = cm[0, 0]
        fp = cm[0, 1]
        fn = cm[1, 0]
        tp = cm[1, 1]
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)
        sensitivity_scores.append(sensitivity)
        specificity_scores.append(specificity)

    return {
        'Accuracy': accuracies.mean(),
        'F1-score': f1_scores.mean(),
        'Sensitivity': np.mean(sensitivity_scores),
        'Specificity': np.mean(specificity_scores)
    }


# Inisialisasi klasifier SVM dan KNN
svm_classifier = SVC()
knn_classifier = KNeighborsClassifier()

# Scenario 1: 10-fold cross-validation
svm_scores = cross_validation(X, y_encoded, svm_classifier)
knn_scores = cross_validation(X, y_encoded, knn_classifier)

# Scenario 2: 75% training, 25% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Scenario 3: 50% training, 25% validation, 25% testing
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.33, random_state=42, stratify=y_train_val)
y_train_encoded = label_encoder.transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# Tampilkan hasilnya
print("Scenario 1: 10-fold Cross-Validation")
print("SVM Accuracy (CV):", svm_scores['Accuracy'])
print("SVM F1-score (CV):", svm_scores['F1-score'])
print("SVM Sensitivity (CV):", svm_scores['Sensitivity'])
print("SVM Specificity (CV):", svm_scores['Specificity'])
print()
print("KNN Accuracy (CV):", knn_scores['Accuracy'])
print("KNN F1-score (CV):", knn_scores['F1-score'])
print("KNN Sensitivity (CV):", knn_scores['Sensitivity'])
print("KNN Specificity (CV):", knn_scores['Specificity'])
print()

print("Scenario 2: 75% Training, 25% Testing")
print("SVM Accuracy (Train):", svm_accuracy_train)
print("SVM F1-score (Train):", svm_f1_train)
print("SVM Sensitivity (Train):", svm_sensitivity_train)
print("SVM Specificity (Train):", svm_specificity_train)
print()
print("KNN Accuracy (Train):", knn_accuracy_train)
print("KNN F1-score (Train):", knn_f1_train)
print("KNN Sensitivity (Train):", knn_sensitivity_train)
print("KNN Specificity (Train):", knn_specificity_train)
print()

print("Scenario 3: 50% Training, 25% Validation, 25% Testing")
print("SVM Accuracy (Train):", svm_accuracy_train_val)
print("SVM F1-score (Train):", svm_f1_train_val)
print("SVM Sensitivity (Train):", svm_sensitivity_train_val)
print("SVM Specificity (Train):", svm_specificity_train_val)
print()
print("KNN Accuracy (Train):", knn_accuracy_train_val)
print("KNN F1-score (Train):", knn_f1_train_val)
print("KNN Sensitivity (Train):", knn_sensitivity_train_val)
print("KNN Specificity (Train):", knn_specificity_train_val)


Scenario 1: 10-fold Cross-Validation
SVM Accuracy (CV): 0.44000000000000006
SVM F1-score (CV): 0.26888888888888884
SVM Sensitivity (CV): 0.0
SVM Specificity (CV): 1.0

KNN Accuracy (CV): 0.6567999999999999
KNN F1-score (CV): 0.6268068008474272
KNN Sensitivity (CV): 0.006666666666666666
KNN Specificity (CV): 0.8045454545454545

Scenario 2: 75% Training, 25% Testing
SVM Accuracy (Train): 0.44
SVM F1-score (Train): 0.2688888888888889
SVM Sensitivity (Train): 0.0
SVM Specificity (Train): 1.0

KNN Accuracy (Train): 0.6368
KNN F1-score (Train): 0.6240663041666368
KNN Sensitivity (Train): 0.08
KNN Specificity (Train): 0.8290909090909091

Scenario 3: 50% Training, 25% Validation, 25% Testing
SVM Accuracy (Train): 0.44
SVM F1-score (Train): 0.2688888888888889
SVM Sensitivity (Train): 0.0
SVM Specificity (Train): 1.0

KNN Accuracy (Train): 0.6048
KNN F1-score (Train): 0.5915752385612919
KNN Sensitivity (Train): 0.2
KNN Specificity (Train): 0.8458646616541353
